In [1]:
from bokeh.plotting import figure
from bokeh.io import push_notebook, output_notebook, show
import math
from bokeh.models.widgets import Select
from bokeh.models.glyphs import Circle
from bokeh.layouts import column
from re import match
import numpy as np
from sklearn import cluster, datasets
from bokeh.models import HoverTool, PanTool, WheelZoomTool, ColumnDataSource
from sklearn.tree import DecisionTreeClassifier
from ipywidgets import interact
import ipywidgets as widgets
import pandas as pd
# from word_cloud_master.wordcloud import wordcloud
import networkx as nx
import matplotlib as mpl
#import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

#self-defined module
import data_parser
import spacy
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
output_notebook()


[nltk_data] Downloading package punkt to /Users/zhiliyang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zhiliyang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Loading BokehJS ...

The codes below read necessary data from csv dataset, and use spacy to generate a document for each movie.

In [2]:

nlp = spacy.load('en')
CHARACTERS_TO_SPLIT = """\[\].,():;-!?\n\""""
with open("data/stopwords.txt", "r") as f:
    STOP_WORDS = set(f.read().strip().split("\n"))
    
for stop_word in STOP_WORDS:
    nlp.vocab[stop_word].is_stop = True
    
for c in CHARACTERS_TO_SPLIT:
    nlp.vocab[c].is_stop = True
    
songs_df = data_parser.parse_csv_data('./data/lyrics.csv', 'ISO-8859-1')
songs_feature_names = data_parser.extract_column_names(songs_df)
#print(songs_feature_names)
songs_dic_list = data_parser.build_row_dic_list(songs_df, songs_feature_names)

In [3]:
artist_list_dic = {}
# Merge the song texts from same artist.
for song in songs_dic_list:
    if song['artist'] in artist_list_dic:
        artist_list_dic[song['artist']]['text'] += song['text']
    else:
        artist_list_dic[song['artist']] = {'artist': song['artist'], 'text': song['text']}
# artist_list = []
# artist_name = ""
# for song in songs_dic_list:
#     if artist_name != song['artist']:
#         artist_list.append(song['text'])
#         artist_name = song['artist']
#     else:
#         artist_list[len(artist_list) - 1] += song['artist']
artist_list_dic = {}
for song in songs_dic_list:
    if song['artist'] in artist_list_dic:
        artist_list_dic[song['artist']].append(song['text'])
    else:
        artist_list_dic[song['artist']] = [song['text']]

artist_song_count_vector_dic = {}
for k, v in artist_list_dic.items():
    count_vect = CountVectorizer(stop_words = 'english')
    song_count = count_vect.fit_transform(v)
    artist_song_count_vector_dic[k] = {'count_vector': count_vect, 'count': song_count}
vo_freq = artist_song_count_vector_dic['ABBA']['count_vector'].vocabulary_
#print(vo_freq)

In [4]:
#print(len(tokenizer.tokenize_words(artist_list_dic['ABBA'])))

In [5]:

artist_list_dic = {}
# Merge the song texts from same artist.
for song in songs_dic_list:
    if song['artist'] in artist_list_dic:
        artist_list_dic[song['artist']]['text'] += song['text']
    else:
        artist_list_dic[song['artist']] = {'artist': song['artist'], 'text': song['text']}


word_freq_dic = {}
for k, v in artist_list_dic.items():
    tok_list = word_tokenize(artist_list_dic[k]['text'])
    filtered_words = [tok for tok in tok_list if tok.lower() not in stopwords.words('english') and tok.lower() not in CHARACTERS_TO_SPLIT and tok.lower() not in STOP_WORDS]
    word_freq = FreqDist(filtered_words)
    word_freq_dic[k] = word_freq

#square_source = ColumnDataSource(data = dict(x = word_freq_dic['ABBA'], y = t_square_attr_list[1]))


In [6]:
from bokeh.palettes import Dark2_5 as palette
from bokeh.transform import factor_cmap
from ipywidgets import interact
from bokeh.layouts import column

keys = [t[0] for t in word_freq_dic['ABBA'].most_common(20)]
values = [t[1] for t in word_freq_dic['ABBA'].most_common(20)]
legends = [str(i)+"-"+str(t[0]) for i, t in enumerate(word_freq_dic['ABBA'].most_common(20))]
freq_plot = figure(plot_width=700, plot_height=700)
freq_source = ColumnDataSource(data = dict(x = range(20), top = values, legend=legends))
freq_plot.vbar(x='x', top='top',source=freq_source, width=0.9, bottom=0, legend="legend")
freq_plot.y_range.start = 0


artists = list(word_freq_dic.keys())

attr0_dropdown = widgets.Dropdown(
    options = artists,
    value = artists[0],
    description = 'Artist',
    disabled=False,
)


    


def update_plot(attr0):
    values = [t[1] for t in word_freq_dic[attr0].most_common(20)]
    legends = [str(i)+"-"+str(t[0]) for i, t in enumerate(word_freq_dic[attr0].most_common(20))]
    freq_source.data['top'] = values
    freq_source.data['legend'] = legends
    push_notebook()


layout = column(freq_plot)
show(layout, notebook_handle = True)

interact(update_plot, attr0 = attr0_dropdown)


A Jupyter Widget

<function __main__.update_plot>